# N-Grams

An N-Gram is an ordered sequence of words. For example:

![ngrams-numbers](images/ngrams-numbers.png)

In the following series of quizes, you will work with 2-grams, or [bigrams](https://en.wikipedia.org/wiki/Bigram), as they are more commonly called.
The objective is to create a function that calculates the probability that a particular sentence
could occur in a corpus of text, based on the probabilities of its component bigrams. We'll do this in stages though:

- Quiz 1 - Extract tokens and bigrams from a sentence
- Quiz 2 - Calculate probabilities for bigrams
- Quiz 3 - Calculate the log probability of a given sentence based on a corpus of text using bigrams


## Quiz 1 - Extract tokens and bigrams from a sentence

In the quiz below, write a function that returns a list of tokens and a list of bigrams for a given sentence. You will need to first break a sentence into words in a list, then add a `<s>` and `<s/>` token to the
start and end of the list to represent the start and end of the sentence.

Your final lists should be in the format shown above and called out in the function doc string.

In [1]:
test_sentences = [
    'the old man spoke to me',
    'me to spoke man old the',
    'old man me old man me',
]

def sentence_to_bigrams(sentence):
    """
    Add start '<s>' and stop '</s>' tags to the sentence and tokenize it into a list
    of lower-case words (sentence_tokens) and bigrams (sentence_bigrams)
    :param sentence: string
    :return: list, list
        sentence_tokens: ordered list of words found in the sentence
        sentence_bigrams: a list of ordered two-word tuples found in the sentence
    """
    #TODO implement
    sentence_tokens = ['<s>'] + sentence.lower().split() + ['</s>']
    
    sentence_bigrams = []
    sentence_bigrams.append([(sentence_tokens[i], sentence_tokens[i+1]) for i in range(len(sentence_tokens)-1)])
    return sentence_tokens, sentence_bigrams

sentence_to_bigrams(test_sentences[0])

(['<s>', 'the', 'old', 'man', 'spoke', 'to', 'me', '</s>'],
 [[('<s>', 'the'),
   ('the', 'old'),
   ('old', 'man'),
   ('man', 'spoke'),
   ('spoke', 'to'),
   ('to', 'me'),
   ('me', '</s>')]])

## Quiz 2 - Calculate Probabilities and Likelihoods with Bigrams

Recall from a previous video that the probability of a series of words can be calculated from the chained probabilities of its history:

![eqn-jointprob-words-in-sentence](images/eqn-jointprob-words-in-sentence.png)


The probabilities of sequence occurrences in a large textual corpus can be calculated this way and used as a language model to add grammar and contectual knowledge to a speech
recognition system. However, there is a prohibitively large number of calculations for all the possible sequences of varying length in a large textual corpus.

To address this problem, we use the [Markov Assumption](https://en.wikipedia.org/wiki/Markov_property) to approximate a sequence probability with a shorter sequence:

![eqn-markov-assumption-ngrams](images/eqn-markov-assumption-ngrams.png)

In the bigram case, the equation reduces to a series of bigram probabilities multiplied together to find the approximate probability for a sentence. A concrete example:

**`P("I Iove language models") ≈ P("love" ∣ "I") P("language" ∣ "love") P("models" ∣ "language") P("love" ∣ "I") P("language" ∣ "love") P("models" ∣ "language")`**

We can calculate the probabilities by using **[counts](https://docs.python.org/3.6/library/collections.html#collections.Counter)** of the bigramsand individual tokens. The counts are represented below with the `c()` operator:

![eqn-bigram-mle](images/eqn-bigram-mle.png)

In [2]:
# https://stackoverflow.com/questions/54962539/how-to-get-the-probability-of-bigrams-in-a-text-of-sentences

In [3]:
### Utils

def bigrams_from_transcript(filename):
    """
    read a file of sentences, adding start '<s>' and stop '</s>' tags; Tokenize it into a list of lower case words
    and bigrams
    :param filename: string 
        filename: path to a text file consisting of lines of non-puncuated text; assume one sentence per line
    :return: list, list
        tokens: ordered list of words found in the file
        bigrams: a list of ordered two-word tuples found in the file
    """
    tokens = []
    bigrams = []
    with open(filename, 'r') as f:
        for line in f:
            line_tokens, line_bigrams = sentence_to_bigrams(line)
            tokens = tokens + line_tokens
            bigrams = bigrams + line_bigrams
    return tokens, bigrams

In [4]:
from collections import Counter

def sample_run():
    # sample usage by test code (this definition not actually run for the quiz)
    tokens, bigrams = bigrams_from_transcript('transcripts.txt')
    bg_dict = bigram_mle(tokens, bigrams)
    print(bg_dict)


def bigram_mle(tokens, bigrams):
    """
    provide a dictionary of probabilities for all bigrams in a corpus of text
    the calculation is based on maximum likelihood estimation and does not include
    any smoothing. 
    A tag '<unk>' has been added for unknown probabilities.
    :param tokens: list
        tokens: list of all tokens in the corpus
    :param bigrams: list
        bigrams: list of all two word tuples in the corpus
    :return: dict
        bg_mle_dict: a dictionary of bigrams:
            key: tuple of two bigram words, in order OR <unk> key
            value: float probability

    """
    bg_mle_dict = {}
    bg_mle_dict['<unk>'] = 0.
    
    #TODO implement
    token_counter = Counter(tokens)
    bigram_raw = []
    for bigram in bigrams:
        bigram_raw += bigram
    bigram_raw_counts = Counter(bigram_raw)
    
    for bg in bigram_raw_counts:
        bg_mle_dict[bg] = bigram_raw_counts[bg] / token_counter[bg[0]]
    return bg_mle_dict

In [5]:
# tokens, bigrams = bigrams_from_transcript('transcripts.txt')
# bigram_mle(tokens, bigrams)

## Quiz 3 - Calculate the log probability of a given sentence based on a corpus of text using bigrams

There are still a couple of problems to sort out before we use the bigram probability dictionary to calculate the probabilities of new sentences:

1. Some possible combinations may not exist in our probability dictionary but are still possible. We don't want to multiply in a probability of 0 just because our original corpus was deficient. This is solved through "smoothing". There are a number of methods for this, but a simple one is the [Laplace smoothing](https://en.wikipedia.org/wiki/Additive_smoothing) with the "add-one" estimate where VV is the size of the vocabulary for the corpus, i.e. the number of unique tokens:


![eqn-addone-bigram-smoothing](images/eqn-addone-bigram-smoothing.png)

2. Repeated multiplications of small probabilities can cause underflow problems in computers when the values become to small. To solve this, we will calculate all probabilities in log space: 

    `**log(p1 × p2 × p3 × p4) = log p1 + log p2 + log p3 + log p4**`

In [6]:
### Utils
import numpy as np

def bigram_add1_logs(transcript_file):
    """
    provide a smoothed log probability dictionary based on a transcript
    :param transcript_file: string
        transcript_file is the path filename containing unpunctuated text sentences
    :return: dict
        bg_add1_log_dict: dictionary of smoothed bigrams log probabilities including
        tags: <s>: start of sentence, </s>: end of sentence, <unk>: unknown placeholder probability
    """

    tokens, bigrams = bigrams_from_transcript(transcript_file)
    token_counts = Counter(tokens)
    
    bigram_raw = []
    for bigram in bigrams:
        bigram_raw += bigram
    bigram_counts = Counter(bigram_raw)
    
    vocab_count = len(token_counts)

    bg_addone_dict = {}
    for bg in bigram_counts:
        bg_addone_dict[bg] = np.log((bigram_counts[bg] + 1.) / (token_counts[bg[0]] + vocab_count))
    bg_addone_dict['<unk>'] = np.log(1. / vocab_count)
    return bg_addone_dict

In [29]:
def sample_run():
    # sample usage by test code (this definition not actually run for the quiz)
    bigram_log_dict = bigram_add1_logs('transcripts.txt')
    for sentence in test_sentences:
        print(f'*** "{sentence}"')
        print(log_prob_of_sentence(sentence, bigram_log_dict))

def log_prob_of_sentence(sentence):
    total_log_prob = 0.

    # TODO implement
    # get the sentence bigrams with sentence_to_bigrams
    # look up the bigrams from the sentence in the bigram_log_dict
    # add all the log probabilities together
    # if a word doesn't exist, be sure to use the value of the '<unk>' lookup instead
    tokens, bigrams = sentence_to_bigrams(sentence)
    bigram_log_dict = bigram_mle(tokens, bigrams)

    for bg in bigrams:
        for i in range(len(bg)):
            if bg[i] in bigram_log_dict:
                total_log_prob += bigram_log_dict[bg[i]]
            else:
                total_log_prob += bigram_log_dict['<unk>']
                
    return total_log_prob

In [33]:
print(test_sentences[0])
log_prob_of_sentence(test_sentences[0])

the old man spoke to me


7.0

In [35]:
print(test_sentences[1])
log_prob_of_sentence(test_sentences[1])

me to spoke man old the


7.0

In [36]:
print(test_sentences[2])
log_prob_of_sentence(test_sentences[2])

old man me old man me


6.0

**more likely sentences yield higher values for the log probabilities.**